<a href="https://colab.research.google.com/github/jnpos/group8_ECG/blob/Classifier/ECG_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/jveenland/tm10007_ml.git

Cloning into 'tm10007_ml'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), 67.99 MiB | 5.63 MiB/s, done.


In [3]:
# General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import metrics
import zipfile
import os
import pandas as pd

# Metrics
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import LinearSVC
from sklearn.metrics import fbeta_score

In [4]:
with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 827
The number of columns: 9001


**Split input, output, test and train data:**

In [5]:
# Find column with label 
bool_cols = [col for col in data 
             if np.isin(data[col].dropna().unique(), [0, 1]).all()]
loc_label = data.columns.get_loc('label')

# Determine data and output 
y = data['label']
x = pd.DataFrame()
x = data.drop(data.columns[loc_label],axis=1)


In [6]:
# Delete patients (samples) with missing data
deleted_patients = []
zero_samples = []
num_zeros = []
zero_counts = []
eenvariabele = []
print(len(x))
for i in range(len(x)):
  num_zeros = (x.iloc[i] == 0).sum()
  if (num_zeros/750 >= 1) & (num_zeros%750 == 0):                   # met deze regel controleer je of het aantal 0'en overeenkomt met het aantal frequentie features dat in 1 lead zit
    deleted_patients.append(i)
    zero_samples.append("Sample_" + str(i))
    zero_counts.append(num_zeros)

x_dropped = x.drop(deleted_patients)
x_dropped = x_dropped.reset_index(drop=True)
y_dropped = y.drop(deleted_patients)
y_dropped = y_dropped.reset_index(drop=True)
table = pd.DataFrame({'Sample ID': zero_samples, 'Zero count': zero_counts})

print(table)
print(f'The number of samples: {len(x_dropped.index)}')
print(f'The number of columns: {len(x_dropped.columns)}')

827
     Sample ID  Zero count
0   Sample_177         750
1   Sample_251         750
2   Sample_269         750
3   Sample_321         750
4   Sample_323         750
5   Sample_385         750
6   Sample_434         750
7   Sample_446         750
8   Sample_537         750
9   Sample_542         750
10  Sample_575         750
11  Sample_601         750
12  Sample_784         750
13  Sample_790         750
The number of samples: 813
The number of columns: 9000


In [7]:
# Split test and trainingsdata 
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_dropped, y_dropped,test_size=0.25,random_state=0,stratify=y_dropped)

**Scaling:**

In [8]:
# SCALEN
scaler = preprocessing.RobustScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train))

# Niet fitten op test, alleen toepassen


**Checking if data is normally distributed:**


In [9]:
# Checking if data is normally distributed  

from scipy.stats import shapiro  
shapiro(x_train)

amount_normallydistributed = 0
for column in x_train.columns:
  result = shapiro(x_train[column])
  normallydistributed = result.pvalue > 0.05
  amount_normallydistributed += normallydistributed 
  
print(amount_normallydistributed, "features are normally distributed features")


/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


0 features are normally distributed features


**Feature selection:**

In [10]:
# Feature selection

# Univariate & f_classif
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif,k="all")
selector.fit(x_train,y_train)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()

dataframe = pd.DataFrame()
dataframe = selector.pvalues_

print(len(selector.pvalues_[selector.pvalues_<0.05]))

fs = SelectKBest(score_func=f_classif, k=len(selector.pvalues_[selector.pvalues_<0.05]))
X_univariate = fs.fit_transform(x_train, y_train)
print(f'result univariate: {X_univariate.shape}')

# Elastic net
from sklearn.linear_model import ElasticNet, Lasso, ElasticNetCV
from sklearn.feature_selection import SelectFromModel

#for i in np.arange(0.02,1,0.02):
#regr = ElasticNetCV(cv=5, random_state=0)
#regr.fit(x_train, y_train)
#print(regr.alpha_)
#print(regr.intercept_)
regr_alpha_ = 0.3858647907662583

ENreg = ElasticNet(alpha=regr_alpha_, l1_ratio=0.5).fit(x_train,y_train)
model_ENreg = SelectFromModel(ENreg, prefit=True)
x_ENreg = model_ENreg.transform(x_train)
#print(f'result Elastic Net: {x_ENreg.shape}, met als i:{i}')
print(f'result Elastic Net: {x_ENreg.shape}')


855
result univariate: (609, 855)
result Elastic Net: (609, 6)


**PCA:**

In [24]:
# PCA
X_univariate = pd.DataFrame(X_univariate)
n_samples = len(X_univariate)
n_features = len(X_univariate.columns)
n_features = min((n_samples//2), n_features)
print(n_features)

p = PCA(n_components=n_features)
p = p.fit(X_univariate)
x_pca = p.transform(X_univariate)

print(f'result pca: {x_pca.shape}')

304
result pca: (609, 304)
304
result pca: (609, 304)


**Optimize hyperparameters of all classifiers:**

LSVM

In [25]:
l_svm = LinearSVC()
#logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              #random_state=0)
distributions = dict(penalty=['l2', 'l1'],
                     loss = ['hinge','squared_hinge'],                                       
                     dual=[False, False],
                     tol = np.geomspace(1e-7, 1e2, num=10),
                     C = np.geomspace(1e-5, 1e5, num=11),
                     fit_intercept=[True, True],
                     intercept_scaling = range(1,10),
                     class_weight = ['balanced','balanced'])
#y_pred = clf.predict(x_train)
#l_svm.fit(x_train, y_train)


#y_pred = l_svm.fit(x_train, y_train).predict(x_train)

clf = RandomizedSearchCV(l_svm, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) #dual = False als n_samples > n_features 
search = clf.fit(x_ENreg, y_train)
print(search.best_params_)




{'tol': 100.0, 'penalty': 'l1', 'loss': 'squared_hinge', 'intercept_scaling': 9, 'fit_intercept': True, 'dual': False, 'class_weight': 'balanced', 'C': 0.01}


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to 0.0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss

{'tol': 0.01, 'penalty': 'l2', 'loss': 'squared_hinge', 'intercept_scaling': 6, 'fit_intercept': True, 'dual': False, 'class_weight': 'balanced', 'C': 0.01}


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to 0.0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, loss

DT

In [26]:
from sklearn import tree
import numpy as np

tree = tree.DecisionTreeClassifier()

distributions = dict(criterion =['gini','entropy','log_loss'],
                     splitter=['best','random'],
                     min_samples_split=range(2,10),
                     min_samples_leaf=range(1,10),
                     min_weight_fraction_leaf = np.linspace(0, 0.5, 25),
                     max_features=['sqrt','log2',None],
                     class_weight=['balanced','balanced'])

clf = RandomizedSearchCV(tree, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) 
search = clf.fit(x_ENreg, y_train)
print(search.best_params_)

{'splitter': 'best', 'min_weight_fraction_leaf': 0.16666666666666666, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': None, 'criterion': 'entropy', 'class_weight': 'balanced'}
{'splitter': 'random', 'min_weight_fraction_leaf': 0.020833333333333332, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None, 'criterion': 'log_loss', 'class_weight': 'balanced'}


SVC

In [ ]:
from sklearn import svm

svm = svm.SVC()

distributions = dict(C = np.geomspace(1e-5, 1e5, num=11),
                     kernel=['linear','poly','rbf','sigmoid'],
                     degree = range(0,5),
                     gamma=['scale','auto'],
                     coef0 = range(-5,5),
                     shrinking = [True,False],
                     probability = [True, False],
                     tol = np.geomspace(1e-7, 1e2, num=10),
                     cache_size=range(10,400),
                     class_weight = ['balanced','balanced'])

clf = RandomizedSearchCV(svm, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) #dual = False als n_samples > n_features 

search = clf.fit(x_ENreg, y_train)
print(search.best_params_)

RF for X_univariate:

In [13]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

distributions = dict(n_estimators = range(5,1000,5),
                     criterion =['gini','entropy','log_loss'],
                     min_samples_split=range(2,10),
                     min_samples_leaf=range(1,10),
                     min_weight_fraction_leaf = np.linspace(0, 0.5, 25),
                     max_features=['sqrt','log2',None],
                     bootstrap=[True,False],
                     oob_score=[True,False],
                     warm_start=[True,False],
                     class_weight=['balanced','balanced'])

clf = RandomizedSearchCV(RF, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) 
search = clf.fit(X_univariate, y_train)
print(search.best_params_)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the cl

{'warm_start': True, 'oob_score': True, 'n_estimators': 555, 'min_weight_fraction_leaf': 0.1875, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_features': None, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': True}


RF for x_ENreg:

In [14]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

distributions = dict(n_estimators = range(5,1000,5),
                     criterion =['gini','entropy','log_loss'],
                     min_samples_split=range(2,10),
                     min_samples_leaf=range(1,10),
                     min_weight_fraction_leaf = np.linspace(0, 0.5, 25),
                     max_features=['sqrt','log2',None],
                     bootstrap=[True,False],
                     oob_score=[True,False],
                     warm_start=[True,False],
                     class_weight=['balanced','balanced'])

clf = RandomizedSearchCV(RF, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) 
search = clf.fit(x_ENreg, y_train)
print(search.best_params_)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the cl

{'warm_start': True, 'oob_score': True, 'n_estimators': 795, 'min_weight_fraction_leaf': 0.14583333333333331, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'log2', 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': True}
